<a href="https://www.quantrocket.com"><img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png"></a><br>
<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

***
[Intraday Futures Calendar Spreads](Introduction.ipynb) › Part 4: Real-Time Data Collection
***

# Real-Time Data Collection

For paper trading we will collect native combo data each day for the calendar spread that interests us. 

## Create real-time database

First, we create a tick db for collecting bid and ask prices. We associate it with the 'cl-fut' universe (because this argument is required) but in reality we will specify a particular combo each time we collect data. Importantly, we specify `primary_exchange=True` to indicate that we want to collect native combo data. (See the usage guide for more on combos.)

In [1]:
from quantrocket.realtime import create_ibkr_tick_db
create_ibkr_tick_db("cl-combo-tick", universes="cl-fut", fields=["BidPrice", "AskPrice"], primary_exchange=True)

{'status': 'successfully created tick database cl-combo-tick'}

Then, we create an aggregate database of 1-minute bars from the tick database. As data is collected into the tick database, it will automatically be aggregated into 1-minute bars in the aggregate database.

In [2]:
from quantrocket.realtime import create_agg_db
create_agg_db("cl-combo-tick-1min", tick_db_code="cl-combo-tick", bar_size="1min", fields={"BidPrice": ["Close"], "AskPrice": ["Close"]})

{'status': 'successfully created aggregate database cl-combo-tick-1min from tick database cl-combo-tick'}

## Collect real-time data

Each day, we want to collect native combo market data for the calendar spread of month 1 and 2 (in this example). The exact contracts which make up this combo will change over time, specifically, on each new rollover date. 

We create a custom script to carry out our data collection strategy. The custom code is provided in a function called `collect_combo` in the file [collect_combo.py](collect_combo.py). The function performs the following steps:

* query the securities master database and identify the current month 1 and month 2 contracts
* create a combo from these contracts (if it doesn't already exist)
* initiate real-time data collection for this combo

### Running the custom code

The custom code can be run by importing the function in Python or at the command line using the satellite service. The advantage of using the satellite service is that we can schedule the command to run daily from our countdown service crontab (see the scheduling notebook). The command is shown below. It can be executed manually for testing purposes by executing the following cell:

In [ ]:
!quantrocket satellite exec 'codeload.calspread.collect_combo.collect_combo' --params 'universe:cl-fut' 'contract_months:[1,2]' 'tick_db:cl-combo-tick' 'until:16:30:00 America/New_York'

***

## *Next Up*

Part 5: [Moonshot Native Spread Strategy](Part5-Moonshot-Native-Spread-Strategy.ipynb)